In [33]:
import glob
import os
import pandas as pd
import numpy as np
import shutil
import librosa
from tqdm import tqdm
from os import listdir

def extract_spectrum(file_name, file_length=5, bin_factor=4, **kwargs):
    """
    file_length is the target time length of the file in seconds
    bin_factor is the amount by which we enlarge the time bins from 512 samples:
    Extract mel spectrogram from audio file `file_name`
    """

    X, sample_rate = librosa.core.load(file_name)
    target_name = file_name[:-4]
    target_length = int(file_length * sample_rate)
    target = np.zeros(target_length)
    if X.shape[0] >= target_length:
        target = X[:target_length]
    else:
        target[:X.shape[0]] = X
    X = target
    mel = librosa.feature.melspectrogram(X, sr=sample_rate, hop_length=512 * bin_factor).T
    return mel


def wav_reprocess(filename):
    file_pro = extract_spectrum(filename, mel=True)
    file_pro = file_pro[:,:,np.newaxis]
    return file_pro

def process_wav_folder_with_labels(wav_folder_path, labels_file_path):
    y = pd.read_csv(labels_file_path)
    os.chdir(wav_folder_path)
    X = []
    files = y['filename'].tolist() 
    for filename in files:
          X.append(wav_reprocess(file_path)) 
    X = np.asarray(X)

    gender_dict = {'female':0, 'male':1}
    y['gender'] = y['gender'].map(gender_dict)
    y = np.asarray(y['gender'], dtype = 'int32')

    return X, y

In [34]:
main_folder = '/content/drive/My Drive/ITC'
wav_folder_path = '/content/drive/My Drive/ITC/waves'
labels_file_path = '/content/drive/My Drive/ITC/label.csv'
X, y = process_wav_folder_with_labels(wav_folder_path, labels_file_path)

In [35]:
X.shape, y.shape

((21, 54, 128, 1), (21,))

In [36]:
os.chdir(main_folder)
np.save('X', X)
np.save('y', y)

In [37]:
xx = np.load('X.npy')
yy = np.load('y.npy')
xx.shape, yy.shape

((21, 54, 128, 1), (21,))